In [1]:
import psutil
from memory_profiler import memory_usage
import pandas as pd
import numpy as np
import time
import joblib
from category_encoders.target_encoder import TargetEncoder
import numpy as np
import onnx
import onnxruntime as ort
import time
import wandb
import os

In [2]:
def _check_usage_of_cpu_and_memory():
    
    pid = os.getpid()
    py  = psutil.Process(pid)
    
    cpu_usage   = os.popen("ps aux | grep " + str(pid) + " | grep -v grep | awk '{print $3}'").read()
    cpu_usage   = cpu_usage.replace("\n","")
    
    memory_usage  = round(py.memory_info()[0] /2.**30, 2)
    
    print("cpu usage\t\t:", cpu_usage, "%")
    print("memory usage\t\t:", memory_usage, "%")

## AdGraph

In [34]:
data_df = pd.read_csv('./dataset/adgraph/ad_features.csv', index_col=[0])
label_df = pd.read_csv('./dataset/adgraph/ad_labelled.csv', index_col=[0])
data_df = data_df.reset_index(drop=True)
label_df = label_df.reset_index(drop=True)
# label_df = label_df['label']
# data_df = pd.concat([data_df, label_df], axis = 1)
df_labelled = data_df.merge(label_df[['visit_id', 'name', 'label']], on=['visit_id', 'name'])
df_labelled = df_labelled[df_labelled['label'] != "Error"]
te = TargetEncoder()
df_labelled['content_policy_type'] = te.fit_transform(df_labelled['content_policy_type'], df_labelled.label)

test = df_labelled[df_labelled.columns[2:-1]].astype('float32')

_check_usage_of_cpu_and_memory()
mem_usage_before = memory_usage(-1, interval=1, timeout=1)

cpu_percent_start = psutil.cpu_percent(interval=1)
adgraph_model_path = '/data/kiho/webtrack/out/ad_graph_model.joblib'
adgraph_model = joblib.load(adgraph_model_path)
start_time = time.time()
tmp = adgraph_model.predict(test)
end_time = time.time()
_check_usage_of_cpu_and_memory()
# time.sleep(1)
# Measure CPU utilization during model's predict method
cpu_percent_during = psutil.cpu_percent(interval=1)

# Get the CPU utilization after running the model
# cpu_after = psutil.cpu_percent(interval=1)
mem_usage_after = memory_usage(-1, interval=1, timeout=1)
print('Memory used:', max(mem_usage_after)-max(mem_usage_before), 'MB')  # Report additional memory used by the model
# Calculate and print the elapsed time
# print('CPU utilization:', cpu_percent_start - cpu_percent_during)

elapsed_time = end_time - start_time
print(f'Inference time: {elapsed_time} seconds')

# Memory used: 12.66796875 MB
# Inference time: 17.83465051651001 seconds

/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1210: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


cpu usage		: 0.1 %
memory usage		: 7.8 %
cpu usage		: 0.1 %
memory usage		: 7.96 %
Memory used: 165.03515625 MB
Inference time: 17.868902444839478 seconds


## WebGraph

In [35]:
data_df = pd.read_csv('./dataset/webgraph/web_features.csv', index_col=[0])
label_df = pd.read_csv('./dataset/webgraph/web_labelled.csv', index_col=[0])
data_df = data_df.reset_index(drop=True)
label_df = label_df.reset_index(drop=True)
# label_df = label_df['label']
# data_df = pd.concat([data_df, label_df], axis = 1)

df_labelled = data_df.merge(label_df[['visit_id', 'name', 'label']], on=['visit_id', 'name'])
df_labelled = df_labelled[df_labelled['label'] != "Error"]

test = df_labelled[df_labelled.columns[2:-1]].astype('float32')

_check_usage_of_cpu_and_memory()
cpu_before = psutil.cpu_percent(interval=1)
mem_usage_before = memory_usage(-1, interval=1, timeout=1)

start_time = time.time()

webgraph_model_path = '/data/kiho/webtrack/out/web_graph_model.joblib'
webgraph_model = joblib.load(webgraph_model_path)

start_time = time.time()

tmp = webgraph_model.predict(test)

end_time = time.time()

# time.sleep(1)
# Measure CPU utilization during model's predict method
# cpu_percent_during = psutil.cpu_percent(interval=1)


_check_usage_of_cpu_and_memory()
# Get the CPU utilization after running the model
cpu_after = psutil.cpu_percent(interval=1)
print('CPU utilization:', cpu_after - cpu_before)
mem_usage_after = memory_usage(-1, interval=1, timeout=1)
print('Memory used:', max(mem_usage_after)-max(mem_usage_before), 'MB')  # Report additional memory used by the model
# Calculate and print the elapsed time

elapsed_time = end_time - start_time
print(f'Inference time: {elapsed_time} seconds')

# cpu usage		: 0.1 %
# memory usage		: 6.81 %
# cpu usage		: 0.1 %
# memory usage		: 7.07 %
# CPU utilization: 0.1999999999999993
# Memory used: 260.94921875 MB
# Inference time: 19.177070379257202 seconds

/data/kiho/mambaforge/envs/autocomplete/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1210: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


cpu usage		: 0.1 %
memory usage		: 8.47 %
cpu usage		: 0.1 %
memory usage		: 8.6 %
CPU utilization: -0.19999999999999996
Memory used: 132.38671875 MB
Inference time: 17.9814236164093 seconds


## De-Adlock

In [33]:
data_df = pd.read_csv('./dataset/trainset.csv', index_col=[0])
test = data_df[data_df.columns[2:-1]]
# Load the ONNX model

cpu_before = psutil.cpu_percent(interval=1)
_check_usage_of_cpu_and_memory()
mem_usage_before = memory_usage(-1, interval=1, timeout=1)



model = onnx.load('./model/De-Adlock.onnx')
# Check that the IR is well formed
onnx.checker.check_model(model)

start_time = time.time()

# Create an ONNX runtime session
ort_session = ort.InferenceSession('./model/De-Adlock.onnx')
input_data = test.values.astype('float32')
input_name = ort_session.get_inputs()[0].name

# Run the inference session to get the prediction results
predictions = ort_session.run(None, {input_name: input_data})

end_time = time.time()

# time.sleep(1)
# Measure CPU utilization during model's predict method
cpu_percent_during = psutil.cpu_percent(interval=1)
_check_usage_of_cpu_and_memory()
mem_usage_after = memory_usage(-1, interval=1, timeout=1)

# Get the CPU utilization after running the model
# cpu_after = psutil.cpu_percent(interval=1)
# print('CPU utilization:', cpu_before - cpu_percent_during)

elapsed_time = end_time - start_time
print(f'Inference time: {elapsed_time} seconds')
print('Memory used:', max(mem_usage_after)-max(mem_usage_before), 'MB')  # Report additional memory used by the model

# cpu usage		: 34.0 %
# memory usage		: 5.74 %
# cpu usage		: 35.8 %
# memory usage		: 5.74 %
# Inference time: 1.3394038677215576 seconds
# Memory used: 0.03515625 MB

cpu usage		: 0.1 %
memory usage		: 7.71 %
cpu usage		: 0.1 %
memory usage		: 7.71 %
Inference time: 1.9481086730957031 seconds
Memory used: -0.10546875 MB


In [32]:
data_df = pd.read_csv('./dataset/trainset.csv', index_col=[0])
test = data_df[data_df.columns[2:-1]]
# Load the ONNX model

cpu_before = psutil.cpu_percent(interval=1)
_check_usage_of_cpu_and_memory()
mem_usage_before = memory_usage(-1, interval=1, timeout=1)



model = onnx.load('./model/De-Adlock.onnx')
# Check that the IR is well formed
onnx.checker.check_model(model)

start_time = time.time()

# Create an ONNX runtime session
ort_session = ort.InferenceSession('./model/De-Adlock.onnx')
input_data = test.values.astype('float32')
input_name = ort_session.get_inputs()[0].name

# Run the inference session to get the prediction results
predictions = ort_session.run(None, {input_name: input_data})

end_time = time.time()

# time.sleep(1)
# Measure CPU utilization during model's predict method
cpu_percent_during = psutil.cpu_percent(interval=1)
_check_usage_of_cpu_and_memory()
mem_usage_after = memory_usage(-1, interval=1, timeout=1)

# Get the CPU utilization after running the model
# cpu_after = psutil.cpu_percent(interval=1)
# print('CPU utilization:', cpu_before - cpu_percent_during)

elapsed_time = end_time - start_time
print(f'Inference time: {elapsed_time} seconds')
print('Memory used:', max(mem_usage_after)-max(mem_usage_before), 'MB')  # Report additional memory used by the model

# cpu usage		: 0.1 %
# memory usage		: 7.71 %
# cpu usage		: 0.1 %
# memory usage		: 7.71 %
# Inference time: 2.3571176528930664 seconds
# Memory used: 0.17578125 MB

cpu usage		: 0.1 %
memory usage		: 7.71 %
cpu usage		: 0.1 %
memory usage		: 7.71 %
Inference time: 2.3571176528930664 seconds
Memory used: 0.17578125 MB
